In [2]:
## TO DO ##

# Change labels
# Add poverty maps

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [4]:
summary = pd.read_csv('https://github.com/ngpsu22/Child_Allowance_States/raw/main/poverty_gini_tax_child_allowance')
summary.drop('Unnamed: 0', 1, inplace = True)

## Tax Reform and Gini

In [5]:
tax = summary[summary['race'] == 'All']
tax = tax[tax['age_group']=='all']

x = tax.child_allowance.unique()
state_names = tax.state.unique()
default_state = state_names[0]

REFORM = {'state_tax_rate': 'State tax rate', 
           'fed_tax_rate': "Federal tax rate"}
GINI = {'fed_gini': 'Gini Index with Federal Tax ',
        'state_gini': 'Gini Index with State Tax ',
        'non_funded_gini': 'Gini Index with No Funding'}

data_list = []
for state in state_names:
    state_data = tax[tax.state==state]
    state_list = []
    for reform in REFORM:
        state_list.append(state_data[reform])
    for gini in GINI:
        state_list.append(state_data[gini])
    data_list.append(state_list)

data_columns = list(REFORM.keys()) + list(GINI.keys())
data = pd.DataFrame(data_list, columns = data_columns)
data['State'] = state_names
data = data.set_index('State')

def getDataList(state):
    data_list = []
    for dc in data_columns:
        data_list.append(data.loc[state][dc])
    return data_list

fig = make_subplots(rows=2, cols=1, 
                    subplot_titles = ('Revenue neutral tax rates in ' + default_state, 
                                      'Gini Index in' + default_state))

for reform in REFORM:
    fig.add_trace(go.Scatter(
        x=x, 
        y=tax[tax.state==default_state][reform],
        name=REFORM[reform]),
        row = 1, col = 1
    )
for gini in GINI:
    fig.add_trace(go.Scatter(
        x=x, 
        y=tax[tax.state==default_state][gini],
        name=GINI[gini]),
        row = 2, col = 1
    )
    
buttons = []
for state in state_names:
    new_button = {'method': 'update',
                  'label': state,
                  'args': [{'y': getDataList(state)},
                            {'annotations[0].text': 'Revenue neutral tax rates in ' + state,
                            'annotations[1].text': 'Gini index in ' + state,}
                          ]}
    buttons.append(new_button)
    
# construct menus
updatemenus = [{'buttons': buttons,
                'direction': 'down',
                'showactive': True,
                'pad':{"r": 50},
               }]

# update layout with buttons, and show the figure
fig.update_layout(updatemenus=updatemenus)

fig.update_xaxes(title_text='Monthly Child Allowance', row=1, col=1)
fig.update_xaxes(title_text='Monthly Child Allowance', row=2, col=1)
fig.update_yaxes(title_text='Tax Rate on Taxable Income', row=1, col=1)
fig.update_yaxes(title_text='Gini Index', row=2, col=1)

fig.update_layout(
        yaxis_ticksuffix='%',
        font=dict(family='Roboto'),
        hovermode='x', 
        xaxis_tickprefix='$',
        xaxis_ticksuffix='',
        plot_bgcolor='white',
        legend_title_text=''   
    )

fig.update_traces(mode='markers+lines', hovertemplate=None)

fig.update_layout(height=600, margin=dict(l=0, r=0, t=100, b=0), title_text="Tax Reform and Gini Index")
fig

## Maps

In [6]:
no_fund_states = summary[summary['age_group'] == 'child']
no_fund_states = summary[summary['race'] == 'All']

no_fund_map = px.choropleth(no_fund_states, 
                       locations='code',
                       color='non_funded_poverty_rate', 
                       animation_frame='child_allowance',
                       color_continuous_scale='reds',
                       locationmode='USA-states',
                       scope='usa',
                        range_color=(0, 22),
                       title=('Child poverty with with no tax reform'),
                       height=600,
                       labels={'child_allowance':'Child Allowance',
                              'code': 'State',
                              'non_funded_poverty_rate': 'Child poverty rate'})

no_fund_map.update_layout(coloraxis_showscale=False)
no_fund_map.show(config={'displayModeBar': False})

In [7]:
fed_states = summary[summary['age_group'] == 'child']
fed_states = summary[summary['race'] == 'All']

fed_map = px.choropleth(fed_states, 
                        locations='code',
                        color='fed_poverty_rate', 
                        animation_frame='child_allowance',
                        color_continuous_scale='reds',
                        locationmode='USA-states',
                        scope='usa',
                        range_color=(0, 22),
                        title=('Child poverty with federally funded tax reform'),
                        height=600,
                        labels={'child_allowance':'Child Allowance',
                                'code': 'State',
                                'fed_poverty_rate': 'Child poverty rate'})

fed_map.update_layout(coloraxis_showscale=False)
fed_map.show(config={'displayModeBar': False})

In [8]:
# create a choropleth map for state funded child allowance

state_states = summary[summary['age_group'] == 'child']
state_states = summary[summary['race'] == 'All']

state_map = px.choropleth(state_states, 
                       locations='code',
                       color='state_poverty_rate', 
                       animation_frame='child_allowance',
                       color_continuous_scale='reds',
                       locationmode='USA-states',
                       scope='usa',
                        range_color=(0, 22),
                       title=('Child poverty with state funded tax reform'),
                       height=600,
                       labels={'child_allowance':'Child Allowance',
                              'code': 'State',
                              'state_poverty_rate': 'Child poverty rate'})

state_map.update_layout(coloraxis_showscale=False)
state_map.show(config={'displayModeBar': False})

## Poverty - Age

In [9]:
age_groups = summary[summary['race'] == 'All']
age_groups

,code,state,child_allowance,age_group,race,non_funded_poverty_rate,non_funded_tax_rate,non_funded_gini,fed_poverty_rate,fed_tax_rate,fed_gini,state_poverty_rate,state_tax_rate,state_gini
1,AL,Alabama,0,child,All,14.8,0.0,0.418,14.8,0.0,0.418,14.8,0.0,0.418
5,AL,Alabama,0,adult,All,12.1,0.0,0.440,12.1,0.0,0.440,12.1,0.0,0.440
9,AL,Alabama,0,all,All,12.7,0.0,0.446,12.7,0.0,0.446,12.7,0.0,0.446
13,AL,Alabama,50,child,All,13.7,0.0,0.409,13.7,0.5,0.408,13.7,0.7,0.408
17,AL,Alabama,50,adult,All,11.8,0.0,0.438,11.8,0.5,0.437,11.8,0.7,0.437
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6845,US,US,450,adult,All,10.2,0.0,0.424,10.5,4.8,0.416,10.5,4.8,0.417
6849,US,US,450,all,All,9.0,0.0,0.418,9.2,4.8,0.410,9.2,4.8,0.411
6853,US,US,500,child,All,4.2,0.0,0.352,4.3,5.3,0.341,4.3,5.4,0.342
6857,US,US,500,adult,All,10.1,0.0,0.422,10.4,5.3,0.414,10.4,5.4,0.414


## Poverty - Race

In [10]:
race = summary[summary['age_group'] == 'child']